In [1]:
from bs4 import BeautifulSoup as bf
import requests
import re

In [2]:
def load_soup(url):
    html = requests.get(url).text
    soup = bf(html, 'html5lib')
    return soup

In [4]:
# builds url lists specific to the site
def Drugs_url_list(drug_stem, pg_init, pg_n):
    url_c = ['https://www.drugs.com/comments/', '/?page=']
    url_list = [url_c[0] + drug_stem + url_c[1] +str(ik) for ik in range(pg_init, pg_n+1)]
    return url_list

def WebMD_url_list(drug_stem, pg_init, pg_n):
    url_c = ['https://www.webmd.com/drugs/drugreview-', '&pageIndex=', '&sortby=3&conditionFilter=-500' ]
    url_list = [url_c[0] + drug_stem + url_c[1] +str(ik) + url_c[2] for ik in range(pg_init, pg_n+1)]
    return url_list
    

In [5]:
# takes url list, scrapes, returns pages of soup
def scraper(url_list):
    soup_list = []
    for url in url_list:
        soup_list.append( [url, load_soup(url)])
    return soup_list 

In [6]:
# uses methods from the two parser classes to slice and dice soup into review objects
def parse_reviews(pages, tag, drug, parser):
    print(len(pages))
    for page in pages:
        rev_stew = page[1].find_all('div' ,{'class' : tag})
        for ik, item in enumerate(rev_stew):
            new_review = review(drug, item, parser, ik)
            drug.reviews.append(new_review)       

In [7]:
# each drug has a name and a url_stem
class drug:
    
    def __init__(self, name, url_stem):
        self.name = name
        self.url_stem = url_stem
        self.reviews = []

In [8]:
# ************ webMD.com ************ Parser

class WebMD:
    
        def __init__(self, name):
            self.name = name

            
        def set_reviewerMeta (_rev_soup, ik):
            try:
                reviewer_temp = _review_soup.find('p', {'class':'reviewerInfo'}).text
                return reviewer_temp.strip('Reviewer: ')
            except:
                return None
            
        #below takes reviewer soup
        def set_userName (self, _reviewerMeta):
            try:
                return _reviewerMeta.split(',')[0]
            except:
                return None

        def set_ageRange (self, _reviewerMeta):
            try:
                return re.search('\s\w+[-]\w+\s', _reviewerMeta).group().strip()
            except:
                return None

        def set_gender (self, _reviewerMeta):
            try: 
                return re.split('\s\w+[-]\w+\s', _reviewerMeta)[1].split()[0]
            except: 
                return None

        def set_medDuration (self, _reviewerMeta):
            try:
                return re.split('on Treatment for ', _reviewerMeta)[1].split('(Patient)')[0].strip()
            except:
                return None
            
        #below takes full soup
        #untested for webMD
        def set_reviewDate  (self, _rev_soup):
            try:
                heading = _reviewerMeta.find('div', {'class': 'postHeading clearfix'}).text
                heading.find('div', {'class': 'date'}).text
            except:
                return None
                
        def set_condition (self, _rev_soup):
            try:
                condition = _rev_soup.find('div', {'class': 'conditionInfo'}).text
                temp = condition.split('Condition: ')[1]
#                 print(temp)
                return temp
            except:
                return None
                
        def set_effectiveness (self, _rev_soup):
                try:
                    temp = _rev_soup.find('div' ,{'class' : 'catRatings firstEl clearfix'}).text
                    return int(re.search(r'\d+', temp).group())
                except:
                    return None

        def set_ease_of_use (self, _rev_soup):
                try:
                    temp = _rev_soup.find('div' ,{'class' : 'catRatings clearfix'}).text
                    return int(re.search(r'\d+', temp).group())
                except:
                    return None

        def set_satisfaction (self, _rev_soup):
                try:
                    temp = _rev_soup.find('div' ,{'class' : 'catRatings lastEl clearfix'}).text
                    return int(re.search(r'\d+', temp).group())
                except:
                    return None
                
        #from drugs.com        
        def set_genRating  (self, _rev_soup):
                    return None

        def set_comment (self, _rev_soup, ik):
                try: 
                    temp = _rev_soup.find('p', {'id':'comFull'+str(ik+1)}).text
                    temp = re.split('Hide Full', temp)[0]
                    return temp.lstrip('Comment:')
                except:
                    return None

        def set_upVotes (self, _rev_soup):
                try:
                    temp = _rev_soup.find('p', {'class' : "helpful"}).text
                    return int(re.search(r'\d+', temp).group())
                except:
                    return None

In [9]:
# ************ Drugs.com ************  Parser

# modified from source: 
# https://blog.nycdatascience.com/student-works/web-scraping/anti-epileptic-drug-review-analysis/

class DrugsDotCom:
    
        def __init__(self, name):
            self.name = name

        # fetch information about author
        def set_reviewerMeta (self, _rev_soup):
                for tag in ['user-name user-type user-type-2_non_member', 'user-name user-type user-type-0_select_member']:
                    try:
                        MetaData = _rev_soup.find('p', {'class': tag}) 
                    except:
                        MetaData = 2
                if MetaData != 2:
                    return MetaData
                else:
                    return 2

                
        def set_userName (self, _reviewerMeta):
                try:
                    return _reviewerMeta.b.get_text()
                except:
                    return None

                
        #need to fix this        
        def set_ageRange (self, _reviewerMeta):
                    try:
                        return re.search('\s\w+[-]\w+\s', _reviewerMeta).group().strip()
                    except:
                        return None
                    
                    
        #gender not specified on drugs.com
        def set_gender (self, _reviewerMeta):
                return None


        def set_medDuration (self, _reviewerMeta):
                try: 
                    return _reviewerMeta.find('span', {'class': 'small light'}).text()
                except:
                    return None

                
        def set_reviewDate (self, _reviewerMeta):
                try:
                    return _reviewerMeta.find('span', {'class': 'light comment-date'}).text()
                except:
                    return None
                
                
        def set_condition (self, _rev_soup):
                try:
                    return _rev_soup.find('div', {'class':'user-comment'}).b.get_text()
                except:
                    return None
                
                
        #from WebMD.com
        def set_effectiveness (self, _rev_soup):
                return None
            
            
        #from WebMD.com
        def set_ease_of_use (self, _rev_soup):
                return None
            
            
        #from WebMD.com
        def set_satisfaction (self, _rev_soup):
                return None

            
        def set_genRating (self, _rev_soup):        
                try:
                    return _rev_soup.find('div',{'class': 'rating-score'}).get_text()
                except:
                    return None


        def set_comment (self, _rev_soup, ik):
                try:
                    return _rev_soup.find('div', {'class':'user-comment'}).span.get_text()
                except:
                    return None


        def set_upVotes (self, _rev_soup):
                try: 
                    temp = _rev_soup.find('p', {'class': 'note'}).b.get_text()
                    return int(re.search(r'\d+', temp).group())
                except:
                    return None


In [10]:
# Review object
class review(drug):
    
        def __init__(self, drug, _review_soup, site, ik):
            
            reviewer_info = site.set_reviewerMeta(_review_soup)
            self.drugName = drug.name
            self.site = site.name
            self.condition = site.set_condition(_review_soup)
            self.reviewDate = site.set_reviewDate(_review_soup)
            self.userName = site.set_userName(reviewer_info) #temp.split(',')[0]
            self.ageRange = site.set_ageRange(reviewer_info) #re.search('\s\w+[-]\w+\s', temp).group().strip()
            self.gender = site.set_gender(reviewer_info) #re.split('\s\w+[-]\w+\s', temp)[1].split()[0]
            self.medDuration = site.set_medDuration(reviewer_info) #re.split('on Treatment for ', temp)[1].split('(Patient)')[0].strip()
            self.effectiveness = site.set_effectiveness(_review_soup)
            self.ease_of_use = site.set_ease_of_use(_review_soup)
            self.satisfaction = site.set_satisfaction(_review_soup)
            self.genRating = site.set_genRating(_review_soup)
            self.comment = site.set_comment(_review_soup, ik)
            self.upVotes = site.set_upVotes(_review_soup)

Two test cases below:
    1.) Abilify with WebMD
    2.) Methuylphenidate with Drugs.com
    
the code uses url_stem and range of pages to generate a url list, then goes and scrapes those pages,
and parses them into reviews

In [31]:
### WebMD Abilify test case

abilify_stem = '64439-Abilify-oral.aspx?drugid=64439&drugname=Abilify-oral'
abilify = drug('abilify', abilify_stem)
abilify_Soup = scraper(WebMD_url_list(abilify.url_stem, 0, 2))

webMD_parser = WebMD('webMD')
webMD_tag = 'userPost'
parse_reviews(abilify_Soup, webMD_tag, abilify, webMD_parser)

In [11]:
# Drugs.com test

meth_stem = 'methylphenidate'
meth = drug('methylphenidate', meth_stem)
meth_Soup = scraper(Drugs_url_list(meth.url_stem, 0, 2))

drugs_tag = 'block-wrap comment-wrap'
drugs_parser = DrugsDotCom('drugsDotCom')
parse_reviews(meth_Soup, drugs_tag, meth, drugs_parser)

In [14]:
# prints review objects for inspection

for reviewx in meth.reviews:
    print(reviewx.__dict__)
    print('----------')

{'drugName': 'methylphenidate', 'site': 'drugsDotCom', 'condition': 'For ADHD:', 'reviewDate': None, 'userName': None, 'ageRange': None, 'gender': None, 'medDuration': None, 'effectiveness': None, 'ease_of_use': None, 'satisfaction': None, 'genRating': None, 'comment': '"I\'m 49 years old and recently diagnosed with ADHD.  My doctor put me on this medication 18mg about a month ago, I feel somewhat better not quite totally better still have trouble staying focused my anxiety is all but gone so thats great.  I\'ve stopped biting my nails and clenching my teeth, but I always heard that you lose weight on this medicine, but I feel like I\'m gaining weight - actually I am, and I don\'t stop eating not big meals but I\'m constantly eating something! even after a meal.  Is that normal? I can\'t find anywhere where it says that you can gain weight..."', 'upVotes': 0}
----------
{'drugName': 'methylphenidate', 'site': 'drugsDotCom', 'condition': 'Concerta (methylphenidate) for ADHD:', 'reviewDa